In [69]:
import ee
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio

In [70]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWgR1-lVa4_dtM9f7Geh5rmC4tznd04neaNaziY8HQTJB-THErOjx98



Successfully saved authorization token.


In [71]:
ee.Initialize()

In [72]:
Rajshahi = ee.FeatureCollection("projects/ee-bobgiezi/assets/Gates/Rajshahi").geometry().bounds()
Kurigram = ee.FeatureCollection("projects/ee-bobgiezi/assets/Gates/Kurigram").geometry().bounds()

In [73]:
geoms = list()
geoms.append(Rajshahi)
geoms.append(Kurigram)

In [74]:
names = ['Rajshahi', 'Kurigram']

In [76]:
for k in range(len(geoms)):
    for year in range(2001, 2022):
        geom = geoms[k]
        name = names[k]
        
        filename = str(Path('DataExport')/name/(str(year)+'.tif'))

        flood_map = modis.dfo(geom, str(year)+"-06-01", str(year)+"-12-31", "standard", '3Day', True)

        flood_map_slope_mask = misc.apply_slope_mask(flood_map, thresh=5)

        perm_water = misc.get_jrc_perm(geom)

        dfo_final = ee.Image(flood_map_slope_mask).addBands(perm_water)

        geemap.ee_export_image(dfo_final,\
                           filename,\
                           scale=250,\
                           crs='EPSG:4326',\
                           region=geom,\
                           file_per_band=False)

Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Rajshahi/2021.tif
Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Kurigram/2021.tif


In [77]:
for name in names:
    for year in range(2021, 2022):
        with rasterio.open(Path('DataExport')/name/(str(year)+'.tif'),'r+') as dst:
            dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])